In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
# Remove stopwords from a list of words
def remove_stopwords(words_list):
    
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    
    new_words_list = []
    for word in words_list:
        word_lower = word.lower()
        if word_lower not in stopwords_list:
            new_words_list.append(word_lower)
    return new_words_list

In [3]:
def get_signature(sense):
    signature = []
    for word in tokenize_sentence(sense.definition()):  # tokenizzo la definizione del synset
        signature.append(word)
    for example in sense.examples():  # tokenizzo ogni esempio del synset
        for word in tokenize_sentence(example):
            signature.append(word)
    return signature  # la signature conterrà tutte le parole presenti nella definizione del senso e negli esempi

In [4]:
# Tokenizza la frase in input e ne affettua anche la lemmatizzazione della sue parole
def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(str(sentence))):
        if tag[1][:2] == "NN":
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.NOUN))
        elif tag[1][:2] == "VB":
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.VERB))
        elif tag[1][:2] == "RB":
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADV))
        elif tag[1][:2] == "JJ":
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADJ))
    return words_list

In [5]:
from collections import Counter
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk

In [6]:
# dato un soggetto cerca l'oggetto della frase rispetto al verbo considerato
def search_obj(sentence, head_verb, pattern):
    for token in sentence:
        if token.head.text == head_verb and token.dep_ == 'dobj' and len(
                pattern) < 2 and token.text != '\n  ' and token.text != '\n':
            dependency2 = token.text, token.tag_, token.head.text, token.dep_
            return dependency2

In [7]:
# dato un oggetto cerca il soggetto della frase rispetto al verbo considerato
def search_subj(sentence, head_verb, pattern):
    for token in sentence:
        if token.head.text == head_verb and token.dep_ == 'nsubj' and len(
                pattern) < 2 and token.text != '\n  ' and token.text != '\n':
            dependency1 = token.text, token.tag_, token.head.text, token.dep_
            return dependency1

In [8]:
# NON UTILIZZATO (però si può usare). Algortimo di Lesk fatto da me
# ottiene risultati diversi perchè è diversa l'implementazione rispetto a quello
# di nltk

# Data una parola ambigua e il contesto in cui la parola si verifica,
# Lesk restituisce un Synset con il maggior numero di parole sovrapposte tra la frase
# di contesto e definizioni diverse da ciascun Synset.
def lesk_algorithm(word, sentence_words):
    if len(wn.synsets(word)) != 0:  # se non esiste in wordnet
        best_sense = wn.synsets(word)[0]
        max_overlap = 0
        context = remove_stopwords(sentence_words)
        for sense in wn.synsets(word):
            signature = remove_stopwords(get_signature(sense))
            overlap = len(list(set(signature) & set(context)))  # overlap
            if overlap > max_overlap:
                max_overlap = overlap
                best_sense = sense
        return best_sense
    else:
        return None

In [9]:
# prende in input un pattern e restituisce un nuovo pattern
# composto come segue: word_super_sense1 , word_super_sense2
# in pratica disambigua i termini in base alla sentence a cui fanno riferimento
def disambiguate_terms(pattern):
    dependency1 = pattern[0]
    dependency2 = pattern[1]

    sentence = pattern[2]

    word1 = dependency1[0]
    word2 = dependency2[0]

    """
    sense1 = lesk_algorithm(word1, tokenize_sentence(sentence))
    sense2 = lesk_algorithm(word2, tokenize_sentence(sentence))
    """
    # disambiguazione dei significati
    sense1 = lesk(tokenize_sentence(sentence), word1, 'n')
    sense2 = lesk(tokenize_sentence(sentence), word2, 'n')

    # nel new poattern ci saranno i supersensi di sense1 e sense2
    if sense1 and sense2:
        new_pattern = sense1._lexname, sense2._lexname
        return new_pattern

    return None

In [10]:
# Restituisce i cluster semantici
def compute_semantic_clusters(patterns):
    new_patterns = []
    for pattern in patterns:
        new_pattern = disambiguate_terms(pattern)
        if new_pattern:
            # supersense1, supersense2
            new_patterns.append(new_pattern)

    # calcolo percentuali rispetto al numero di nuovi pattern trovati
    results = dict(Counter(new_patterns))
    semantic_clusters = []
    for key in results.keys():
        result = results[key]
        percentage = result / len(new_patterns)
        cluster = key, format(percentage * 100, '.2f') + '%'
        semantic_clusters.append(cluster)

    # ordino in maniera decrescente in base alla frequenza(percentuale)
    semantic_clusters = sorted(semantic_clusters, key=lambda x: x[1], reverse=True)
    return semantic_clusters

In [11]:
import spacy
from xml.dom import minidom

In [12]:

VERB = "buy"
CORPUS = "buy_corpus.xml"
all_verb_forms = ['buy', 'buys']

"""
VERB = "eat"
CORPUS = "eat_corpus.xml"
all_verb_forms = ['eat', 'eats']
"""
ARGUMENTS_N = 2  # numero di argomenti del verbo (VALENZA)

In [13]:
def main():
    nlp = spacy.load('en_core_web_sm')

    mydoc = minidom.parse(CORPUS)

    # trova tutte le frasi del corpus
    sentences = mydoc.getElementsByTagName('line')

    print("Verbo: ", VERB)
    print("Forme del verbo: ", all_verb_forms)

    patterns = []
    for item in sentences:
        # nlp effettua un parsing che restituisce una struttura ad albero a dipendenze
        sentence = nlp(item.firstChild.data.replace("<s>", "").replace("</s>", "").replace("     ", ""))
        pattern = []
        for token in sentence:
            # se trova il soggetto cerca l'oggetto
            if token.head.text in all_verb_forms and token.dep_ == 'nsubj' and len(
                    pattern) < ARGUMENTS_N and token.text != '\n  ' and token.text != '\n':
                dependency1 = token.text, token.tag_, token.head.text, token.dep_
                pattern.append(dependency1)
                dependency2 = search_obj(sentence, token.head.text, pattern)
                if dependency2:
                    pattern.append(dependency2)
        if len(pattern) == ARGUMENTS_N:  # se ho trovato tutti e due gli argomenti (soggetto, oggetto) allora va bene
            pattern.append(sentence)
            patterns.append(pattern)

    print("Numero di pattern trovati: ", len(patterns))
    # calcolo i cluster semantici con le loro percentuali di frequenza
    semantic_clusters = compute_semantic_clusters(patterns)
    for cluster in semantic_clusters:
        print(cluster)


main()

Verbo:  buy
Forme del verbo:  ['buy', 'buys']
Numero di pattern trovati:  2683
(('noun.quantity', 'noun.cognition'), '7.48%')
(('noun.group', 'noun.artifact'), '4.61%')
(('noun.quantity', 'noun.communication'), '3.89%')
(('noun.person', 'noun.artifact'), '3.59%')
(('noun.act', 'noun.substance'), '2.66%')
(('noun.group', 'noun.communication'), '2.25%')
(('noun.substance', 'noun.artifact'), '2.25%')
(('noun.quantity', 'noun.artifact'), '11.68%')
(('noun.quantity', 'noun.person'), '1.95%')
(('noun.quantity', 'noun.act'), '1.84%')
(('noun.person', 'noun.cognition'), '1.84%')
(('noun.quantity', 'noun.food'), '1.74%')
(('noun.quantity', 'noun.attribute'), '1.64%')
(('noun.artifact', 'noun.artifact'), '1.43%')
(('noun.person', 'noun.communication'), '1.43%')
(('noun.quantity', 'noun.quantity'), '1.33%')
(('noun.communication', 'noun.artifact'), '1.33%')
(('noun.person', 'noun.attribute'), '1.23%')
(('noun.cognition', 'noun.artifact'), '1.23%')
(('noun.location', 'noun.artifact'), '1.13%')
(('